<a href="https://colab.research.google.com/github/twofacauth/gensim/blob/master/kunugi_text_service_prod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ヘルプ

## モデル構築

### インプット

- 学習用テキスト

### アウトプット

- 学習済みモデル
    - word2vec: テキストのベクトル化
    - XGBoost: 分類

### 操作の流れ

1. GCSに学習データのアップロード
2. GCS上のファイル確認
3. ファイル名を指定してGCSからColab環境に読み込む
4. 学習

## モデル更新

### インプット

- 学習用テキスト
- 更新前学習済みモデル
    - word2vec: テキストのベクトル化
    - XGBoost: 分類

### アウトプット

- 更新後学習済みモデル

### 操作の流れ

1. GCSに追加学習用のテキストをアップロード
2. GCS上のファイル確認
3. ファイル名を指定してGCSからColab環境に読み込む
4. 追加学習

## 分類

### インプット

- 分類対象テキスト
- 学習済みモデル
    - word2vec: テキストのベクトル化
    - XGBoost: 分類

### アウトプット

- 分類結果

### 操作の流れ

1. GCSに分類対象データのアップロード
2. GCS上のファイル確認
3. ファイル名を指定してGCSからColab環境に読み込む
4. 分類
5. ダウンロード

## ファイルとディレクトリ

GCSバケット名: `prod-kunugi-text-analysis`

### ディレクトリ構成

- `gs://prod-kunugi-text-service/ジャンル名/model/`: モデルを格納
    - `gs://prod-kunugi-text-service/ジャンル名/model/classification/`: 分類モデルのファイルを格納
    - `gs://prod-kunugi-text-service/ジャンル名/model/embedding/`: word2vecなど分散表現を取得するモデルを格納
- `gs://prod-kunugi-text-service/ジャンル名/data/`: 学習や予測に使うテキストファイルを格納
    - `gs://prod-kunugi-text-service/ジャンル名/data/predict/`: 分類対象のデータを格納
    - `gs://prod-kunugi-text-service/ジャンル名/data/pred_result/`: 分類済みのデータを格納
    - `gs://prod-kunugi-text-service/ジャンル名/data/train/`: 学習に使うデータ（教師データ）を格納

### 処理で使うテキストファイル

- カンマ区切りテキスト
- ヘッダなし
- 値（文など）にカンマを含む場合に使う引用符: `"`
- 文字コードはUTF8
- 改行コードはLF+CR
- カラム
    1. URL
    2. 文
    3. NGフラグ（整数型で1=NG / 0=OK）。学習データでのみ必要


# プログラム

## 初期設定

In [ ]:
%%sh
#@title OS環境構築
sudo apt install mecab libmecab-dev mecab-ipadic-utf8 file -y
mkdir -p src
cd src
git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
cd mecab-ipadic-neologd/
sudo ./bin/install-mecab-ipadic-neologd -n -a -y
sudo pip3 install mecab-python3 neologdn
sed -i 's#dicdir = /var/lib/mecab/dic/debian#dicdir = /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd#' /etc/mecabrc

pip install fsspec

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libmagic-mgc libmagic1 libmecab2 mecab-ipadic mecab-jumandic
  mecab-jumandic-utf8 mecab-utils
The following NEW packages will be installed:
  file libmagic-mgc libmagic1 libmecab-dev libmecab2 mecab mecab-ipadic
  mecab-ipadic-utf8 mecab-jumandic mecab-jumandic-utf8 mecab-utils
0 upgraded, 11 newly installed, 0 to remove and 39 not upgraded.
Need to get 29.3 MB of archives.
After this operation, 282 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic-mgc amd64 1:5.32-2ubuntu0.4 [184 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic1 amd64 1:5.32-2ubuntu0.4 [68.6 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 file amd64 1:5.32-2ubuntu0.4 [22.1 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libmecab2 amd64 0.996-5 [257 kB]



debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 11.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Cloning into 'mecab-ipadic-neologd'...
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 11.6M  100 11.6M    0     0  3821k      0  0:00:03  0:00:03 --:--:-- 5156k
./../../libexec/iconv_euc_to_utf8.sh ./Noun.adverbal.csv 
./../../libexec/iconv_euc_to_utf8.sh ./Postp-col.csv 
./../../libexec/iconv_euc_to_utf8.sh ./Interjection.csv 
./../../libexec/iconv_eu

In [11]:
# @title Pythonライブラリ
# 全体で使うライブラリ
import multiprocessing
from gensim.models import word2vec
import gc
from time import process_time
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn import metrics
from datetime import datetime
from google.colab import files
import zipfile


# ユーザ定義関数
## テキスト前処理
import neologdn
import pandas as pd
import numpy as np
import os
import MeCab

def _wakati(sentence, tagger):
  word_classes = []
  root_forms = []
  for chunk in tagger.parse(sentence).splitlines()[:-1]:
    if chunk != '':
      (surface, features) = chunk.split('\t')
      feature = features.split(',')
      # ['品詞', '品詞細分類1', '品詞細分類2', '品詞細分類3', '活用型', '活用形', '原形', '読み', '発音']
      if feature[0] in ['名詞', '動詞', '形容詞', '副詞', '連体詞'] and feature[6] not in ['ある', 'いる', 'する', 'こと', 'れる', 'できる', 'なる', 'の', 'られる', 'ない', '*', 'HTTPS', 'HTTP', 'IMAGE', 'ORGANIZATION', 'Pretty', 'URL', 'WP', 'headline', 'http://', 'https://', 'id', 'jpg', 'PNG', 'gif', 'name', 'object', '0V', 'a', 'JPG', 'PNG', 'GIF', 'WWW']:
        word_classes.append(feature[0])
        root_forms.append(feature[6])
  return pd.Series([word_classes, root_forms])

def _preprocess_text_main(df, col_text, col_word_class='word_classes', col_root_form='root_forms'):
  os.environ['MECABRC'] = '/etc/mecabrc'
  tagger = MeCab.Tagger('-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')
  body = df[col_text].str.strip() # 最初と最後のスペースや改行は削除
  body = body.str.lower()
  body = body.astype(str).apply(neologdn.normalize)
  #body = body.str.replace('[?。]', '\n').str.strip() # 句点で文を分けて改行で区切る（末尾の改行は削除）
  body = body.str.replace('[ \t\f]+', '')
  body = body.str.replace('\n+', '\n') # 連続する改行はまとめる
  body = body.str.replace('\d+(\.\d+)?', '0') # 数字は統一
  df = df.assign(sentence=body.str.split('\n')).explode('sentence') # 1行1文
  df = df.reset_index(drop=True)
  df[[col_word_class, col_root_form]] = df.sentence.apply(_wakati, tagger=tagger)
  return df

def preprocess_text_1col(file):
  with open(file, 'r', encoding='UTF-8') as f:
    df = pd.DataFrame({'body': [f.read()]})
  #df = pd.read_csv(file, header=None)
  #df.rename(columns={0: 'body'}, inplace=True)
  df = _preprocess_text_main(df, 'body')
  #df.to_pickle('woshiru.pickle')
  return df

def preprocess_text_2col(file):
  df = pd.read_csv(file)
  df.columns = ['url', 'body']
  df = _preprocess_text_main(df, 'body')
  #df.to_pickle('woshiru.pickle')
  return df

def sentence_vector(sentence, model):
  v_words = []
  for word in sentence:
    if word in model.wv:
      v_word = model.wv.get_vector(word)
      normlized_v_word = v_word / np.sqrt(np.sum(v_word**2))
      v_words.append(normlized_v_word)
  return(np.mean(v_words, axis=0))

## 機械学習の基本ツール
from itertools import product
def expand_grid(dictionary):
   return pd.DataFrame([row for row in product(*dictionary.values())], columns=dictionary.keys())

## 学習
def train_word2vec(processed_df, params, category):
  # word2vecモデル構築
  word_list = processed_df.root_forms.tolist()
  print('Started building word2vec.')
  time_start = process_time()
  model = word2vec.Word2Vec(word_list, workers=multiprocessing.cpu_count(), **params)
  print('Finished building word2vec: ' + str(process_time() - time_start) + ' sec')

  # word2vecモデル保存
  print('Started saving word2vec model.')
  time_start = process_time()
  model_name = f"w2v_base_{params['sg']}_{params['size']}_{str(params['alpha']).replace('.', '')}_{params['window']}_{params['min_count']}_{params['iter']}"
  model_file_name = f'{model_name}.model'
  model.save(model_file_name)
  model_file_gspath = f'gs://prod-kunugi-text-service/{category}/model/embedding/{model_file_name}'
  !gsutil cp $model_file_name $model_file_gspath
  print('Finished saving word2vec model: ' + str(process_time() - time_start) + ' sec')
  return(model, model_name, model_file_gspath)

def train_xgboost(processed_df, params, word2vec_model, name, category):
  # 分類データ生成
  print('Started generating data to classify.')
  time_start = process_time()
  features_df = processed_df.root_forms.apply(sentence_vector, model=word2vec_model).apply(pd.Series)
  df_to_classify = pd.concat([processed_df['ng'], features_df], axis=1)
  train_dmat = xgb.DMatrix(df_to_classify.drop('ng', axis=1), label=df_to_classify['ng'])
  print('Finished generating data to classify: ' + str(process_time() - time_start) + ' sec')
  
  # 分類の学習
  print('Started training xgboost.')
  time_start = process_time()
  xgb_cv = xgb.cv(
    dtrain = train_dmat, 
    num_boost_round = 10000, 
    nfold = 5,
    early_stopping_rounds = 20, # if specified, see result$best_iteration
    params = params)
  best_iter = xgb_cv.index[xgb_cv['test-auc-mean']==xgb_cv['test-auc-mean'].max()][0]
  xgb_model = xgb.train(
    dtrain = train_dmat,
    num_boost_round = best_iter,
    params = params
  )
  print('Finished training xgboost: ' + str(process_time() - time_start) + ' sec')
  print('Started saving xgboost model.')
  time_start = process_time()
  model_file_name = f'xgb_for_{name}.model'
  xgb_model.save_model(model_file_name)
  model_file_gspath = f'gs://prod-kunugi-text-service/{category}/model/classification/{model_file_name}'
  !gsutil cp $model_file_name $model_file_gspath
  print('Finished saving xgboost model: ' + str(process_time() - time_start) + ' sec')
  return(xgb_model, model_file_gspath)

In [ ]:
#@title 初期化
from google.colab import auth
auth.authenticate_user()

!gcloud config set project seismic-honor-199018

import os
#default_dir = os.getcwd()
#work_dir_name = 'a' # @param{type:"string"}
#work_dir_abspath = os.path.join(default_dir, work_dir_name)
work_dir_abspath = '/content/a'
os.makedirs(work_dir_abspath, exist_ok=True)
os.chdir(work_dir_abspath)

category = 'cardloan' # @param{type:"string"} ['cardloan', 'datsumo']

Updated property [core/project].


In [ ]:
# @title モデリング手法のパラメータ設定
params_word2vec = {
  'sg': 0,
  'size': 200,
  'alpha': 0.025,
  'window': 10,
  'min_count': 3,
  'iter': 100
}

params_xgboost = {
  'booster': 'gbtree', # 'gbtree', 'gblinear', or 'dart'. 基本的に'gbtree'でいい
  'objective': 'binary:logistic',
  'eval_metric': 'auc',
  'eta': 0.1, # 各ステップの学習幅。値が小さいと学習が進みにくい一方で過学習を防げる。0.01～0.2の値になることが多いらしい※。デフォルトで0.3。
  'gamma': 0, # （木の複雑さ）損失関数の減少がこの値を超える時にのみ分割を進める。指定する場合、損失関数によって値が異なるので気を付ける。デフォルトで0
  'max_depth': 3, # （木の複雑さ）最大の木の深さ。値が大きいと木が複雑になり過学習が発生しやすくなる。3～10の値になることが多いらしい※デフォルトで6
  'min_child_weight': 1, # （木の複雑さ）分割中にノードの重みがこれを超えた時点で分割をやめる。値が大きいと木がシンプルになり、過学習しにくくなる。デフォルトで1
#  'subsample': 5/6, # （サンプリング）各ステップの木を作る際に用いるレコードの数をサンプリングする割合。サンプリングすると過学習を防げる。0.5～1の値になることが多いらしい※。デフォルトで1（サンプリングしない）
#  'colsample_bytree': 1/2, # （サンプリング）各ステップの木を作る際に用いる列の数をサンプリングする割合。サンプリングすると過学習を防げる（データにfactor型を含む場合は微妙？）。0.5～1の値になることが多いらしい※。デフォルトで1（サンプリングしない）
  'alpha': 0, # （正規化）L1正規化項の重み。デフォルトで0
  'lambda': 1, # （正規化）L2正規化項の重み。デフォルトで1
  'tree_method': 'hist' # 'gpu_hist', 'hist' or 'exact'
}

In [ ]:
# @title ファイル一覧を確認するコマンド
!gsutil ls gs://prod-kunugi-text-service/**

gs://prod-kunugi-text-service/cardloan/
gs://prod-kunugi-text-service/cardloan/data/
gs://prod-kunugi-text-service/cardloan/data/pred_result/分類済みファイル名.csv.gz
gs://prod-kunugi-text-service/cardloan/data/predict/
gs://prod-kunugi-text-service/cardloan/data/predict/text-20220312214453.csv.gz
gs://prod-kunugi-text-service/cardloan/data/train/
gs://prod-kunugi-text-service/cardloan/data/train/train.csv.gz
gs://prod-kunugi-text-service/cardloan/model/classification/xgb_for_w2v_base_0_200_0025_10_3_100.model
gs://prod-kunugi-text-service/cardloan/model/embedding/w2v_base_0_200_0025_10_3_100.model


## モデル構築

In [ ]:
# @title 学習用データ一覧
# @markdown 学習対象のファイルはこちらに格納してください `gs://prod-kunugi-text-service/{category}/data/train/`
!echo '========================================'
!echo '学習用テキスト（training_file_name）'
target_url = f'gs://prod-kunugi-text-service/{category}/data/train/?*'
!gsutil ls $target_url | xargs -n1 basename
!echo '========================================'

学習用テキスト（training_file_name）
train.csv.gz


In [ ]:
# @title ファイル読込
training_file_name = 'train.csv.gz' # @param{type:"string"}
training_file_gcspath = os.path.join(f'gs://prod-kunugi-text-service/{category}/data/train/', training_file_name)
#training_file_gcspath = 'gs://prod-kunugi-text-service/cardloan/data/train/train.csv.gz' # @param{type:"string"}

!gsutil cp $training_file_gcspath .

basename = os.path.basename(training_file_gcspath)
name, ext = os.path.splitext(basename)
if ext == '.gz':
  !gzip -d -f $basename
  training_file_name = name
else:
  training_file_name = basename

df_train = pd.read_csv(training_file_name, header=None, names=['url', 'sentence', 'ng'], dtype={'url': str, 'sentence': str, 'ng': int}, encoding='utf-8-sig')

Copying gs://prod-kunugi-text-service/cardloan/data/train/train.csv.gz...
- [1 files][826.6 KiB/826.6 KiB]                                                
Operation completed over 1 objects/826.6 KiB.                                    


In [ ]:
#@title 学習（モデル構築）
processed_df_train = _preprocess_text_main(df_train, 'sentence')
w2v_model, name, w2v_model_file_gcspath = train_word2vec(processed_df_train, params_word2vec, category)
xgb_model, xgb_model_file_gcspath = train_xgboost(processed_df_train, params_xgboost, w2v_model, name, category)
print(f'========================================')
print(f'Word2vec モデルファイルのGCSパス')
print(w2v_model_file_gcspath)
print(f'XGBoost モデルファイルのGCSパス')
print(xgb_model_file_gcspath)
print(f'========================================')
#datetime.now().strftime('%Y%m%d%H%M%S')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: FutureWarning: The default value of regex will change from True to False in a future version.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: FutureWarning: The default value of regex will change from True to False in a future version.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: FutureWarning: The default value of regex will change from True to False in a future version.


Started building word2vec.
Finished building word2vec: 72.090090287 sec
Started saving word2vec model.
Copying file://w2v_base_0_200_0025_10_3_100.model [Content-Type=application/octet-stream]...
-
Operation completed over 1 objects/9.0 MiB.                                      
Finished saving word2vec model: 0.13179462300000466 sec
Started generating data to classify.


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


Finished generating data to classify: 5.281096740999999 sec
Started training xgboost.
Finished training xgboost: 452.061266018 sec
Started saving xgboost model.
Copying file://xgb_for_w2v_base_0_200_0025_10_3_100.model [Content-Type=application/octet-stream]...
-
Operation completed over 1 objects/154.0 KiB.                                    
Finished saving xgboost model: 0.1577134860000342 sec
Word2vec モデルファイルのGCSパス
gs://prod-kunugi-text-service/cardloan/model/embedding/w2v_base_0_200_0025_10_3_100.model
XGBoost モデルファイルのGCSパス
gs://prod-kunugi-text-service/cardloan/model/classification/xgb_for_w2v_base_0_200_0025_10_3_100.model


## モデル更新

In [ ]:
# @title モデル更新で使うデータ一覧
!echo '========================================'
!echo '追加学習用テキスト'
target_url = f'gs://prod-kunugi-text-service/{category}/data/predict/?*'
!gsutil ls $target_url | xargs -n1 basename

!echo '========================================'
!echo '元のword2vecモデルファイル'
target_url = f'gs://prod-kunugi-text-service/{category}/model/embedding/?*'
!gsutil ls $target_url | xargs -n1 basename
!echo '========================================'

In [ ]:
# @title ファイル読み込み
## 分類対象データを取得
train_file_name = 'text-20220312214453.csv.gz' # @param{type:"string"}
train_file_gcspath = os.path.join(f'gs://prod-kunugi-text-service/{category}/data/train/', train_file_name)

!gsutil cp $train_file_gcspath .

basename = os.path.basename(train_file_gcspath)
name, ext = os.path.splitext(basename)
if ext == '.gz':
  !gzip -d -f $basename
  train_file_name = name
else:
  train_file_name = basename

df_train = pd.read_csv(train_file_name, header=None, names=['url', 'sentence', 'ng'], dtype={'url': str, 'sentence': str, 'ng': int}, encoding='utf-8-sig')

## word2vecモデルファイルを取得
word2vec_file_name = 'w2v_base_0_200_0025_10_3_100.model' # @param{type:"string"}
word2vec_file_gcspath = os.path.join(f'gs://prod-kunugi-text-service/{category}/model/embedding/', word2vec_file_name)

!gsutil cp $word2vec_file_gcspath .

word2vec_file_name = os.path.basename(word2vec_file_gcspath)
word2vec_model_name, ext = os.path.splitext(word2vec_file_name)
w2v_model = word2vec.Word2Vec.load(word2vec_file_name)


## 分類（予測）

In [ ]:
# @title 分類で使うデータ一覧
!echo '========================================'
!echo '予測用テキスト（predicted_file_name）'
target_url = f'gs://prod-kunugi-text-service/{category}/data/predict/?*'
!gsutil ls $target_url | xargs -n1 basename

!echo '========================================'
!echo 'word2vecモデルファイル（word2vec_file_name）'
target_url = f'gs://prod-kunugi-text-service/{category}/model/embedding/?*'
!gsutil ls $target_url | xargs -n1 basename
!echo '========================================'

予測用テキスト（predicted_file_name）
text-20220312214453.csv.gz
word2vecモデルファイル（word2vec_file_name）
w2v_base_0_200_0025_10_3_100.model


In [ ]:
# @title ファイル読み込み
## 分類対象データを取得
predicted_file_name = 'text-20220312214453.csv.gz' # @param{type:"string"}
predicted_file_gcspath = os.path.join(f'gs://prod-kunugi-text-service/{category}/data/predict/', predicted_file_name)
#predicted_file_gcspath = 'gs://prod-kunugi-text-service/cardloan/data/predict/text-20220312214453.csv.gz' # @param{type:"string"}

!gsutil cp $predicted_file_gcspath .

basename = os.path.basename(predicted_file_gcspath)
name, ext = os.path.splitext(basename)
if ext == '.gz':
  !gzip -d -f $basename
  predicted_file_name = name
else:
  predicted_file_name = basename

df_predict = pd.read_csv(predicted_file_name, header=None, names=['url', 'sentence'], dtype={'url': str, 'sentence': str}, encoding='utf-8-sig')

## word2vecモデルファイルを取得
word2vec_file_name = 'w2v_base_0_200_0025_10_3_100.model' # @param{type:"string"}
word2vec_file_gcspath = os.path.join(f'gs://prod-kunugi-text-service/{category}/model/embedding/', word2vec_file_name)
#word2vec_file_gcspath = 'gs://prod-kunugi-text-service/cardloan/model/embedding/w2v_base_0_200_0025_10_3_100.model' # @param{type:"string"}

!gsutil cp $word2vec_file_gcspath .

word2vec_file_name = os.path.basename(word2vec_file_gcspath)
word2vec_model_name, ext = os.path.splitext(word2vec_file_name)
w2v_model = word2vec.Word2Vec.load(word2vec_file_name)

## 対応するXGBoostモデルファイルを取得
xgboost_file_name = f'xgb_for_{word2vec_model_name}.model'
xgboost_file_gcspath = f'gs://prod-kunugi-text-service/cardloan/model/classification/{xgboost_file_name}'

!gsutil cp $xgboost_file_gcspath .

xgb_model = xgb.Booster()
xgb_model.load_model(xgboost_file_name)

In [14]:
# @title 予測
predicted_result_file_name = '分類済みファイル名.csv' # @param{type:"string"}
save_in_gcs = True #@param {type:"boolean"}

#predicted_result_file_name_compressed = predicted_result_file_name + '.gz'
name, ext = os.path.splitext(predicted_result_file_name)
predicted_result_file_name_compressed = name + '.zip'

processed_df_predict = _preprocess_text_main(df_predict, 'sentence')
features_df = processed_df_predict.root_forms.apply(sentence_vector, model=w2v_model).apply(pd.Series)
predict_dmat = xgb.DMatrix(features_df)
predicted = xgb_model.predict(predict_dmat)
predicted_df = pd.concat([df_predict, pd.Series(predicted, name='predicted')], axis=1)

#predicted_df.sort_values('predicted', ascending=False).to_csv(predicted_result_file_name_compressed, mode='w', header=False, index=False, encoding='utf-8-sig', compression={'method': 'gzip', 'compresslevel': 9})
predicted_df.sort_values('predicted', ascending=False).to_csv(predicted_result_file_name, mode='w', header=False, index=False, encoding='utf-8-sig')

with zipfile.ZipFile(predicted_result_file_name_compressed, 'w', compression=zipfile.ZIP_DEFLATED, compresslevel=9) as z:
  z.write(predicted_result_file_name, arcname=predicted_result_file_name)

files.download(predicted_result_file_name_compressed)

if save_in_gcs == True:
  target_url = f'gs://prod-kunugi-text-service/{category}/data/pred_result/'
  !gsutil cp $predicted_result_file_name $target_url

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: FutureWarning: The default value of regex will change from True to False in a future version.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: FutureWarning: The default value of regex will change from True to False in a future version.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: FutureWarning: The default value of regex will change from True to False in a future version.
/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Copying file://分類済みファイル名.csv [Content-Type=text/csv]...
-
Operation completed over 1 objects/22.3 MiB.                                     


# 残骸

In [ ]:
#@title 元ファイルを正しいフォーマットに調整
!gsutil cp gs://kunugi-text-analysis/**/result_1112坂本.csv .
df = pd.read_csv('result_1112坂本.csv')
df = df.loc[(df.score==0)|(df.score==1), ['url', 'sentence', 'score']].astype({'url': str, 'sentence': str, 'score': int}).reset_index(drop=True)
df.to_csv('train.csv', mode='w', header=False, index=False, encoding='utf-8-sig')
!gzip -9 train.csv
!gsutil cp train.csv.gz gs://prod-kunugi-text-service/cardloan/data/train/

Copying gs://kunugi-text-analysis/data_202201/result_1112坂本.csv...
/ [1 files][  8.6 MiB/  8.6 MiB]                                                
Operation completed over 1 objects/8.6 MiB.                                      
Copying file://train.csv.gz [Content-Type=text/csv]...
/ [1 files][826.6 KiB/826.6 KiB]                                                
Operation completed over 1 objects/826.6 KiB.                                    
